In [ ]:
#!/usr/bin/env python3
"""
Cliente multi-thread para fazer requisições HTTP concorrentes.
"""

import threading
import requests
import time
import argparse
from typing import List, Optional
from dataclasses import dataclass


@dataclass
class RequestResult:
    """Resultado de uma requisição HTTP."""
    thread_id: int
    status_code: int
    response_time: float
    success: bool
    error_message: Optional[str] = None


class MultiThreadClient:
    """Cliente que executa requisições HTTP usando múltiplas threads."""
    
    def __init__(self, num_threads: int, url: str = "http://192.168.242.134:7071/primes?count=1000"):
        """
        Inicializa o cliente multi-thread.
        
        Args:
            num_threads: Número de threads a serem criadas
            url: URL para fazer as requisições (default: http://192.168.242.134:7071/primes?count=1000)
        """
        self.num_threads = num_threads
        self.url = url
        self.results: List[RequestResult] = []
        self.results_lock = threading.Lock()
        
    def make_request(self, thread_id: int) -> None:
        """
        Executa uma requisição HTTP em uma thread específica.
        
        Args:
            thread_id: ID da thread atual
        """
        start_time = time.time()
        
        try:
            print(f"Thread {thread_id}: Iniciando requisição para {self.url}")
            response = requests.get(self.url, timeout=30)
            end_time = time.time()
            response_time = end_time - start_time
            
            result = RequestResult(
                thread_id=thread_id,
                status_code=response.status_code,
                response_time=response_time,
                success=response.status_code == 200
            )
            
            print(f"Thread {thread_id}: Concluída - Status: {response.status_code}, Tempo: {response_time:.2f}s")
            
        except requests.exceptions.RequestException as e:
            end_time = time.time()
            response_time = end_time - start_time
            
            result = RequestResult(
                thread_id=thread_id,
                status_code=-1,
                response_time=response_time,
                success=False,
                error_message=str(e)
            )
            
            print(f"Thread {thread_id}: Erro - {e}")
        
        # Thread-safe append dos resultados
        with self.results_lock:
            self.results.append(result)
    
    def run(self) -> List[RequestResult]:
        """
        Executa todas as threads e aguarda a conclusão.
        
        Returns:
            Lista com os resultados de todas as requisições
        """
        print(f"Iniciando {self.num_threads} threads para requisições em {self.url}")
        print("-" * 60)
        
        threads = []
        start_time = time.time()
        
        # Cria e inicia todas as threads
        for i in range(self.num_threads):
            thread = threading.Thread(target=self.make_request, args=(i + 1,))
            threads.append(thread)
            thread.start()
        
        # Aguarda todas as threads terminarem
        for thread in threads:
            thread.join()
        
        end_time = time.time()
        total_time = end_time - start_time
        
        self._print_summary(total_time)
        return self.results
    
    def _print_summary(self, total_time: float) -> None:
        """Imprime um resumo dos resultados."""
        print("-" * 60)
        print("RESUMO DOS RESULTADOS:")
        print(f"Total de threads: {self.num_threads}")
        print(f"Tempo total de execução: {total_time:.2f}s")
        
        successful_requests = sum(1 for r in self.results if r.success)
        failed_requests = len(self.results) - successful_requests
        
        print(f"Requisições bem-sucedidas: {successful_requests}")
        print(f"Requisições falharam: {failed_requests}")
        
        if self.results:
            avg_response_time = sum(r.response_time for r in self.results) / len(self.results)
            min_response_time = min(r.response_time for r in self.results)
            max_response_time = max(r.response_time for r in self.results)
            
            print(f"Tempo médio de resposta: {avg_response_time:.2f}s")
            print(f"Tempo mínimo de resposta: {min_response_time:.2f}s")
            print(f"Tempo máximo de resposta: {max_response_time:.2f}s")
        
        print("-" * 60)


def main():
    """Função principal do programa."""
    parser = argparse.ArgumentParser(description="Cliente multi-thread para requisições HTTP")
    parser.add_argument(
        "num_threads", 
        type=int, 
        help="Número de threads a serem criadas"
    )
    parser.add_argument(
        "--url", 
        default="http://192.168.242.134:7071/primes?count=1000",
        help="URL para fazer as requisições (default: http://192.168.242.134:7071/primes?count=1000)"
    )
    
    args = parser.parse_args()
    
    if args.num_threads <= 0:
        print("Erro: O número de threads deve ser maior que 0")
        return
    
    # Cria e executa o cliente
    client = MultiThreadClient(args.num_threads, args.url)
    results = client.run()
    
    # Opcionalmente, você pode processar os resultados aqui
    # Por exemplo, salvar em um arquivo, fazer análises adicionais, etc.


if __name__ == "__main__":
    num_threads = 100
    url = "http://localhost:30071/primes?count=1000"
    client = MultiThreadClient(num_threads, url)
    results = client.run()
    #main()

Iniciando 100 threads para requisições em http://192.168.242.134:7071/primes?count=1000
------------------------------------------------------------
Thread 1: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 2: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 3: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 4: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 1: Concluída - Status: 200, Tempo: 0.02s
Thread 5: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 6: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 7: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 8: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 2: Concluída - Status: 200, Tempo: 0.03s
Thread 9: Iniciando requisição para http://192.168.242.134:7071/primes?count=1000
Thread 10: Inicia